In [15]:
%%capture
pip install -r requirements.txt

In [1]:
# Install required packages if not already installed
# You may need to uncomment and run these commands in a separate cell or terminal
# !pip install plotly  # Install the Plotly library for visualizations
# !pip install ipywidgets  # Install the ipywidgets library for interactive widgets
# !jupyter nbextension enable --py widgetsnbextension  # Enable the Jupyter notebook widget extension

import pandas as pd
import plotly.graph_objects as go  # Import Plotly graph objects for creating visualizations
from ipywidgets import widgets  # Import ipywidgets for interactive widgets in Jupyter
from IPython.display import display  # Import display to show widgets in Jupyter
import glob  # Import glob to search for files with specific patterns

# Ensure that Plotly displays correctly in Jupyter Notebook
import plotly.io as pio
pio.renderers.default = 'notebook'  # Set the default renderer to 'notebook' for displaying Plotly in Jupyter

# Read two daily ledgers from CSV files with a specific pattern
daily_ledger_files = sorted(glob.glob('daily_ledger_*.csv'))  # Find all files matching 'daily_ledger_*.csv' and sort them
if len(daily_ledger_files) >= 2:
    # Read the two latest daily ledger files
    daily_ledger_1 = pd.read_csv(daily_ledger_files[-1])  # Read the latest ledger file
    daily_ledger_2 = pd.read_csv(daily_ledger_files[-2])  # Read the second latest ledger file
else:
    # If not enough files are found, print a message and initialize empty DataFrames
    print("Not enough daily ledger files found.")
    daily_ledger_1 = pd.DataFrame()  # Empty DataFrame if no files found
    daily_ledger_2 = pd.DataFrame()  # Empty DataFrame if no files found

# Read the other CSV files for additional financial data
large_exposures = pd.read_csv('large_exposures.csv')  # Read large exposures data
monthly_balance = pd.read_csv('monthly_balance_sheet.csv')  # Read monthly balance sheet data
monthly_income = pd.read_csv('monthly_income_statement.csv')  # Read monthly income statement data
suspicious_transactions = pd.read_csv('suspicious_transactions.csv')  # Read suspicious transactions data

# Function to create a Plotly table figure
def create_plotly_table(df, title):
    if df.empty:
        # Return an empty figure if the DataFrame is empty
        return go.Figure()
    # Create a Plotly table with the DataFrame's columns and values
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),  # Set the table header to the DataFrame's column names
                    fill_color='paleturquoise',  # Set the header color
                    align='left'),  # Align header text to the left
        cells=dict(values=[df[col].tolist() for col in df.columns],  # Set table cells to DataFrame values
                   fill_color='lavender',  # Set cell color
                   align='left'))  # Align cell text to the left
    ])
    fig.update_layout(title=title)  # Add a title to the table
    return fig  # Return the table figure

# Create figures for each dataset by calling the function 'create_plotly_table'
fig_daily_ledger_1 = create_plotly_table(daily_ledger_1, 'Daily Ledger 1')  # Create a figure for the first daily ledger
fig_daily_ledger_2 = create_plotly_table(daily_ledger_2, 'Daily Ledger 2')  # Create a figure for the second daily ledger
fig_large_exposures = create_plotly_table(large_exposures, 'Large Exposures')  # Create a figure for large exposures
fig_monthly_balance = create_plotly_table(monthly_balance, 'Monthly Balance Sheet')  # Create a figure for the balance sheet
fig_monthly_income = create_plotly_table(monthly_income, 'Monthly Income Statement')  # Create a figure for the income statement
fig_suspicious_transactions = create_plotly_table(suspicious_transactions, 'Suspicious Transactions')  # Create a figure for suspicious transactions

# Create tabs to display the figures interactively
tab_contents = ['Daily Ledger 1', 'Daily Ledger 2', 'Large Exposures', 'Monthly Balance Sheet', 'Monthly Income Statement', 'Suspicious Transactions']
# List of titles for each tab corresponding to the data

children = []  # Initialize an empty list to store the widget content for each tab

# For each figure, create a corresponding widget to display the figure
for fig in [fig_daily_ledger_1, fig_daily_ledger_2, fig_large_exposures, fig_monthly_balance, fig_monthly_income, fig_suspicious_transactions]:
    # Each child is a FigureWidget containing the figure
    fig_widget = go.FigureWidget(fig)
    # Add the figure widget to the list of children
    children.append(widgets.VBox([fig_widget]))

# Create the Tab widget
tab = widgets.Tab()
tab.children = children  # Set the children of the tab (one for each dataset)

# Set the title for each tab based on the dataset's name
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])  # Assign each tab a title from the 'tab_contents' list

# Display the tab widget in the Jupyter notebook
display(tab)


    'data': [{'cells': {'align': 'left',
                        '…

In [2]:
import sqlite3  # Import the sqlite3 library to interact with SQLite databases
import pandas as pd  # Import pandas for working with data in DataFrame format

# Connect to the SQLite database (bank_simulation.db)
# This will either create the database if it doesn't exist or connect to it if it does
conn = sqlite3.connect('bank_simulation.db')

# SQL query to list all the tables in the database
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query to get the table names and store the result in a pandas DataFrame
tables = pd.read_sql_query(tables_query, conn)
print("Tables in the database:")  # Print a message indicating that the tables are being listed
print(tables)  # Print the DataFrame showing all tables in the database

# Read data from a specific table ('clients') using a SQL query
clients_df = pd.read_sql_query("SELECT * FROM clients", conn)  # Read the entire 'clients' table into a DataFrame
print("Clients Table:")  # Print a message indicating the content of the 'clients' table is being shown
print(clients_df)  # Print the DataFrame containing the 'clients' table data

# You can also read data from other tables like 'transactions' or 'ledger_entries'
transactions_df = pd.read_sql_query("SELECT * FROM transactions", conn)  # Read the 'transactions' table
ledger_entries_df = pd.read_sql_query("SELECT * FROM ledger_entries", conn)  # Read the 'ledger_entries' table

# Close the connection to the database to free up resources
conn.close()

# Display the data (Optional: If in Jupyter, it'll show the data nicely)
clients_df.head()  # Display the first few rows of the 'clients' table


Tables in the database:
              name
0          clients
1   ledger_entries
2  sqlite_sequence
3     transactions
Clients Table:
   client_id           name account_type currency   balance
0          1    Alice Rossi      savings      EUR  -1997.69
1          2      Bob Verdi     checking      USD  -2218.89
2          3  Carla Bianchi    corporate      EUR  18350.74
3          4     David Nero      savings      GBP   2671.79
4          5        Eva Blu     checking      EUR   5442.28


,client_id,name,account_type,currency,balance
0,1,Alice Rossi,savings,EUR,-1997.69
1,2,Bob Verdi,checking,USD,-2218.89
2,3,Carla Bianchi,corporate,EUR,18350.74
3,4,David Nero,savings,GBP,2671.79
4,5,Eva Blu,checking,EUR,5442.28
